In [1]:

from bs4 import  BeautifulSoup
import json
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [2]:
data = pd.read_csv('Data/liverpool03_15_23.csv', index_col=0, low_memory=False)

In [3]:
con = sqlite3.connect("Database/opta_event_data22_23.db")
cur = con.cursor()


In [4]:
data

,eventId,minute,second,teamId,x,y,expandedMinute,period,type,outcomeType,...,Venue,xThreat,xThreat_gen,playerName,is_open_play,is_progressive,is_pass_into_box,won_possession,is_carry_into_box,progressive_carry
0,2.0,0.0,0.0,26,0.0,0.0,0.0,1.0,NaN,Successful,...,Away,NaN,NaN,Unknown,True,False,NaN,False,NaN,NaN
1,2.0,0.0,0.0,52,0.0,0.0,0.0,1.0,NaN,Successful,...,Home,NaN,NaN,Unknown,True,False,NaN,False,NaN,NaN
2,3.0,0.0,0.0,26,50.0,50.0,0.0,1.0,NaN,Successful,...,Away,-0.004436,0.000000,Mohamed Salah,True,False,NaN,False,NaN,NaN
3,3.5,0.0,1.5,26,30.6,41.0,0.0,1.0,"{'value': 99, 'displayName': 'Carry'}",Successful,...,NaN,0.000757,0.000757,Ibrahima Konaté,True,False,NaN,False,NaN,NaN
4,4.0,0.0,3.0,26,34.9,36.0,0.0,1.0,NaN,Unsuccessful,...,Away,NaN,NaN,Ibrahima Konaté,True,False,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,11531.0,51.0,NaN,52,66.8,70.3,54.0,2.0,NaN,Unsuccessful,...,Home,NaN,NaN,Vinícius Júnior,True,False,NaN,False,NaN,NaN
1891,910.0,0.0,0.0,52,0.0,0.0,7.0,NaN,NaN,Successful,...,Home,NaN,NaN,Unknown,True,False,NaN,False,NaN,NaN
1892,836.0,0.0,0.0,26,0.0,0.0,7.0,NaN,NaN,Successful,...,Away,NaN,NaN,Unknown,True,False,NaN,False,NaN,NaN
1893,1.0,0.0,0.0,52,0.0,0.0,0.0,NaN,NaN,Successful,...,Home,NaN,NaN,Unknown,True,False,NaN,False,NaN,NaN


In [5]:
# convert the column of lists to a string representation
data['satisfiedEventsTypes'] = data['satisfiedEventsTypes'].apply(str)

# save the dataframe to a table in the database
data.to_sql(name='Matches', con=con , if_exists="append")
con.commit()
con.close()


In [6]:
data.match_id

0       1694372
1       1694372
2       1694372
3       1694372
4       1694372
         ...   
1890    1694372
1891    1694372
1892    1694372
1893    1694372
1894    1694372
Name: match_id, Length: 1895, dtype: int64

In [9]:
con = sqlite3.connect("Database/opta_event_data22_23.db")
cur = con.cursor()


In [10]:
pd.read_sql("Select * from Matches Where match_id = 1694372", con)

,level_0,Unnamed: 0,eventId,minute,second,teamId,x,y,expandedMinute,period,...,Venue,xThreat,xThreat_gen,playerName,is_open_play,is_progressive,is_pass_into_box,won_possession,is_carry_into_box,progressive_carry
0,0,None,2.0,0.0,0.0,26,0.0,0.0,0.0,1.0,...,Away,NaN,NaN,Unknown,1,0,NaN,0,NaN,NaN
1,1,None,2.0,0.0,0.0,52,0.0,0.0,0.0,1.0,...,Home,NaN,NaN,Unknown,1,0,NaN,0,NaN,NaN
2,2,None,3.0,0.0,0.0,26,50.0,50.0,0.0,1.0,...,Away,-0.004436,0.000000,Mohamed Salah,1,0,NaN,0,NaN,NaN
3,3,None,3.5,0.0,1.5,26,30.6,41.0,0.0,1.0,...,None,0.000757,0.000757,Ibrahima Konaté,1,0,NaN,0,NaN,NaN
4,4,None,4.0,0.0,3.0,26,34.9,36.0,0.0,1.0,...,Away,NaN,NaN,Ibrahima Konaté,1,0,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,1890,None,11531.0,51.0,NaN,52,66.8,70.3,54.0,2.0,...,Home,NaN,NaN,Vinícius Júnior,1,0,NaN,0,NaN,NaN
1891,1891,None,910.0,0.0,0.0,52,0.0,0.0,7.0,NaN,...,Home,NaN,NaN,Unknown,1,0,NaN,0,NaN,NaN
1892,1892,None,836.0,0.0,0.0,26,0.0,0.0,7.0,NaN,...,Away,NaN,NaN,Unknown,1,0,NaN,0,NaN,NaN
1893,1893,None,1.0,0.0,0.0,52,0.0,0.0,0.0,NaN,...,Home,NaN,NaN,Unknown,1,0,NaN,0,NaN,NaN
